In [1]:
import numpy as np 
import pandas as pd

In [2]:
raw_dataset = pd.read_csv('1940s_movies.csv')

## EDA

In [7]:
raw_dataset.head(250)

,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
0,يوم سعيد,15 يناير 1940,???????,125.0,(محمد كمال) شاب بسيط يهوى الموسيقى والغناء، وي...,"محمد كريم,عبدالوارث عسر,عباس علام","محمد عبدالوهاب,علوية جميل,فاطمة محجوب,فاتن حما...",أفلام عبدالوهاب,"محمد عبدالعظيم,جورج بنوا,بريما فيرا,مصطفى حسن",محمد كريم,حسين رستم,NaN,"محمد عبدالوهاب,أحمد رامي,عزت الهجين,بشارة الخو...",محمد كريم,NaN
1,الباشمقاول,18 يناير 1940,??????,85.0,"تدور أحداث الفيلم حول المحامي ""ممتاز"" الذي يهو...","بديع خيري,توجو مزراحي (أحمد المشرقي)","فوزي الجزايرلي,ميمي شكيب,إحسان الجزايرلي,زوزو ...","توجو مزراحي (أحمد المشرقي),عبدالحميد زكي","محمود نصر,عبدالحليم نصر",NaN,NaN,NaN,فريد غصن,"توجو مزراحي (أحمد المشرقي),إبراهيم حلمي",منتخبات بهنا فيلم
2,تحت السلاح,8 فبراير 1940,NaN,101.4,أمينة فتاة من أسرة عريقة، تقرر العائلة زواجها ...,NaN,"أحمد علام,زوزو شكيب,عباس فارس,محمد الديب,زينات...",ألفيزى اورفانيللى,NaN,NaN,NaN,NaN,NaN,فؤاد الجزايرلي,NaN
3,قلب المرأة,6 مايو 1940,???????,87.0,تدور أحداث الفيلم حول (خيرية)، الفتاة الثرية ا...,توجو مزراحي (أحمد المشرقي),"سليمان نجيب,أمينة رزق,دولت أبيض,عقيلة راتب,سلو...",NaN,عبدالحليم نصر,NaN,NaN,NaN,NaN,توجو مزراحي (أحمد المشرقي),منتخبات بهنا فيلم
4,فتاة متمردة,18 مايو 1940,NaN,110.0,درية طالبة فى مدرسة داخلية، تربطها علاقة حب بش...,أحمد جلال,"ماري كويني,محسن سرحان,أنور وجدي,عباس فارس,بديع...",لوتس فيلم (آسيا وشركاها),"عبدالمنعم صبحى,توليو كيارينى",ماري كويني,NaN,NaN,"يوسف صالح,عبدالعزيز سلام,فريد غصن,ماري كويني,ش...","أحمد جلال,محمد زكى محمود",منتخبات بهنا فيلم
5,رجل بين امراتين,7 اكتوبر 1940,NaN,95.0,يتعرف رافت الرسام على امال عبد الشافى وهو يصطا...,السيد زيادة,"بدر لاما,أمينة رزق,بدرية رأفت,فردوس محمد,عبدال...",كوندور فيلم,عبدالله الشيب,"ألبير نجيب,بدر لاما","ستوديو لاما,محروس زيادة",NaN,"يوسف صالح,سامي نعسان,أمينة رزق,بدر لاما,زوزو ش...",إبراهيم لاما,منتخبات بهنا فيلم
6,صرخة في الليل,31 اكتوبر 1940,NaN,110.0,لا تحفظ الزوجة الساقطة شرف زوجها الثرى فترتبط ...,السيد زيادة,"بدر لاما,رجاء عبده,سميحة سميح,بشارة واكيم,منسى...",كوندور فيلم,إبراهيم شيبا,"بدر لاما,ألبير نجيب",محروس زيادة,NaN,"فريد غصن,يوسف صالح",إبراهيم لاما,منتخبات بهنا فيلم
7,الورشة,28 نوفمبر 1940,??ا??,105.0,يسافر الأسطى (علي عبدالرحمن) صاحب ورشة الميكان...,"عزيزة أمير,محمود ذو الفقار","عزيزة أمير,محمود ذو الفقار,أنور وجدي,نجمة إبرا...",إيزيس فيلم,"أرام ماراليان,فيرى فاركاش (فرانسوا فاركاش)",NaN,محمد كامل,NaN,عبدالحميد عبدالرحمن,إستيفان روستي,منتخبات بهنا فيلم
8,أصحاب العقول,1940,NaN,101.4,كتب أحد المستكشفين الأوربيين إلى صديقه حمدي بك...,NaN,"بهيجة المهدي,بشارة واكيم,فوزي منيب,سليمان نجيب",افلام وادى النيل (فهمي داود وشركاه),ألفيزى اورفانيللى,NaN,NaN,NaN,NaN,ألفيزى اورفانيللى,NaN
9,حياة الظلام,1940,??ا??,94.0,عائلة متوسطة الحال مكونة من زوج وزوجة وابنهما ...,"محمود كامل حسن,أحمد بدرخان","ميمي شكيب,محسن سرحان,روحية خالد,فردوس محمد,علي...",محمد جمال الدين رفعت,محمد عبدالعظيم,جمال مدكور,"علي عابد,روبرت شارفنبرج",NaN,"بيرم التونسي,عبدالحميد عبدالرحمن,محمد الكحلاوي...","أحمد بدرخان,فريد الجندي",NaN


In [3]:
for column in raw_dataset.columns:
    print('Column Name: ', str(column))
    print(raw_dataset[column].describe())

Column Name:  اسم الفيلم
count             331
unique            330
top       خاتم سليمان
freq                2
Name: اسم الفيلم, dtype: object
Column Name:   تاريخ العرض
count      331
unique     216
top       1946
freq        24
Name:  تاريخ العرض, dtype: object
Column Name:   تصنيف الفيلم
count       127
unique        9
top       ??ا??
freq         46
Name:  تصنيف الفيلم, dtype: object
Column Name:   مدة الفيلم (دقيقة)
count    331.000000
mean     106.399698
std       11.097976
min       70.000000
25%      100.000000
50%      105.000000
75%      110.000000
max      152.000000
Name:  مدة الفيلم (دقيقة), dtype: float64
Column Name:   ملخص
count                                                   316
unique                                                  316
top       يربط الحب بين "مديحة" ابنة "صادق باشا" والممثل...
freq                                                      1
Name:  ملخص, dtype: object
Column Name:   تأليف
count           308
unique          206
top       يوسف وهبي
fre

In [342]:
df.loc[df['اسم الفيلم'] == 'ضربة القدر']

,اسم الفيلم,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,اليوم,الشهر,السنة
183,ضربة القدر,??ا??,105.0,بعد تورط ليلى (ليلى مراد) في علاقة غير شرعية م...,"إميل زولا,يوسف وهبي","يوسف وهبي,ليلى مراد,يحيى شاهين,لولا صدقي,زوزو ...",عبدالحميد زكي,"سامى بريل,حسن داهش",عبدالحميد عبدالفتاح,"أنطوان بوليزويس,عثمان حسين",NaN,"محمد فوزي,رياض السنباطي,محمد القصبجي",كمال بركات,NaN,13,يناير,1947


In [5]:
raw_dataset.iloc[:,2] = raw_dataset.iloc[:,2].astype('str')

In [6]:
for genre_entry in raw_dataset.iloc[:,2]:
    if '??' in genre_entry or genre_entry == 'nan':
        genre_entry = np.NaN

#### Create a copy of raw_dataset to apply modifications

In [7]:
date_expanded_dataset = raw_dataset.copy(deep = 'True')

####  Seperate Movie Release Data into Day, Month, Year
    

In [8]:
row_numbers = raw_dataset.shape[0]

In [9]:
date_expanded_dataset[['اليوم', 'الشهر', 'السنة']] = date_expanded_dataset[' تاريخ العرض'].str.split(' ', expand = True)
    

In [10]:
date_expanded_dataset = date_expanded_dataset.drop(columns=[' تاريخ العرض'])

In [11]:
for row in range(date_expanded_dataset.shape[0]):
    release_day = date_expanded_dataset.iloc[row, 14]
    if len(release_day) == 4:
        date_expanded_dataset.iat[row,16] = release_day
        date_expanded_dataset.iat[row,15] = 'يناير'
        date_expanded_dataset.iat[row,14] = 1

In [152]:
date_expanded_dataset.head(240)

,اسم الفيلم,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,اليوم,الشهر,السنة
0,يوم سعيد,???????,125.0,(محمد كمال) شاب بسيط يهوى الموسيقى والغناء، وي...,"محمد كريم,عبدالوارث عسر,عباس علام","محمد عبدالوهاب,علوية جميل,فاطمة محجوب,فاتن حما...",أفلام عبدالوهاب,"محمد عبدالعظيم,جورج بنوا,بريما فيرا,مصطفى حسن",محمد كريم,حسين رستم,NaN,"محمد عبدالوهاب,أحمد رامي,عزت الهجين,بشارة الخو...",محمد كريم,NaN,15,يناير,1940
1,الباشمقاول,??????,85.0,"تدور أحداث الفيلم حول المحامي ""ممتاز"" الذي يهو...","بديع خيري,توجو مزراحي (أحمد المشرقي)","فوزي الجزايرلي,ميمي شكيب,إحسان الجزايرلي,زوزو ...","توجو مزراحي (أحمد المشرقي),عبدالحميد زكي","محمود نصر,عبدالحليم نصر",NaN,NaN,NaN,فريد غصن,"توجو مزراحي (أحمد المشرقي),إبراهيم حلمي",منتخبات بهنا فيلم,18,يناير,1940
2,تحت السلاح,nan,101.4,أمينة فتاة من أسرة عريقة، تقرر العائلة زواجها ...,NaN,"أحمد علام,زوزو شكيب,عباس فارس,محمد الديب,زينات...",ألفيزى اورفانيللى,NaN,NaN,NaN,NaN,NaN,فؤاد الجزايرلي,NaN,8,فبراير,1940
3,قلب المرأة,???????,87.0,تدور أحداث الفيلم حول (خيرية)، الفتاة الثرية ا...,توجو مزراحي (أحمد المشرقي),"سليمان نجيب,أمينة رزق,دولت أبيض,عقيلة راتب,سلو...",NaN,عبدالحليم نصر,NaN,NaN,NaN,NaN,توجو مزراحي (أحمد المشرقي),منتخبات بهنا فيلم,6,مايو,1940
4,فتاة متمردة,nan,110.0,درية طالبة فى مدرسة داخلية، تربطها علاقة حب بش...,أحمد جلال,"ماري كويني,محسن سرحان,أنور وجدي,عباس فارس,بديع...",لوتس فيلم (آسيا وشركاها),"عبدالمنعم صبحى,توليو كيارينى",ماري كويني,NaN,NaN,"يوسف صالح,عبدالعزيز سلام,فريد غصن,ماري كويني,ش...","أحمد جلال,محمد زكى محمود",منتخبات بهنا فيلم,18,مايو,1940
5,رجل بين امراتين,nan,95.0,يتعرف رافت الرسام على امال عبد الشافى وهو يصطا...,السيد زيادة,"بدر لاما,أمينة رزق,بدرية رأفت,فردوس محمد,عبدال...",كوندور فيلم,عبدالله الشيب,"ألبير نجيب,بدر لاما","ستوديو لاما,محروس زيادة",NaN,"يوسف صالح,سامي نعسان,أمينة رزق,بدر لاما,زوزو ش...",إبراهيم لاما,منتخبات بهنا فيلم,7,اكتوبر,1940
6,صرخة في الليل,nan,110.0,لا تحفظ الزوجة الساقطة شرف زوجها الثرى فترتبط ...,السيد زيادة,"بدر لاما,رجاء عبده,سميحة سميح,بشارة واكيم,منسى...",كوندور فيلم,إبراهيم شيبا,"بدر لاما,ألبير نجيب",محروس زيادة,NaN,"فريد غصن,يوسف صالح",إبراهيم لاما,منتخبات بهنا فيلم,31,اكتوبر,1940
7,الورشة,??ا??,105.0,يسافر الأسطى (علي عبدالرحمن) صاحب ورشة الميكان...,"عزيزة أمير,محمود ذو الفقار","عزيزة أمير,محمود ذو الفقار,أنور وجدي,نجمة إبرا...",إيزيس فيلم,"أرام ماراليان,فيرى فاركاش (فرانسوا فاركاش)",NaN,محمد كامل,NaN,عبدالحميد عبدالرحمن,إستيفان روستي,منتخبات بهنا فيلم,28,نوفمبر,1940
8,أصحاب العقول,nan,101.4,كتب أحد المستكشفين الأوربيين إلى صديقه حمدي بك...,NaN,"بهيجة المهدي,بشارة واكيم,فوزي منيب,سليمان نجيب",افلام وادى النيل (فهمي داود وشركاه),ألفيزى اورفانيللى,NaN,NaN,NaN,NaN,ألفيزى اورفانيللى,NaN,1,يناير,1940
9,حياة الظلام,??ا??,94.0,عائلة متوسطة الحال مكونة من زوج وزوجة وابنهما ...,"محمود كامل حسن,أحمد بدرخان","ميمي شكيب,محسن سرحان,روحية خالد,فردوس محمد,علي...",محمد جمال الدين رفعت,محمد عبدالعظيم,جمال مدكور,"علي عابد,روبرت شارفنبرج",NaN,"بيرم التونسي,عبدالحميد عبدالرحمن,محمد الكحلاوي...","أحمد بدرخان,فريد الجندي",NaN,1,يناير,1940


In [189]:
from googlesearch import search

In [213]:
for row in range(row_numbers):
    search_input = 'elcinema '
    search_input += date_expanded_dataset.iloc[row,0] 
    search_input += ' '
    search_input += date_expanded_dataset.iloc[row,16]
        
    link = None
    for search_result in search(search_input, tld="co.in", num=1, stop=1, pause=2): 
        link = search_result
    
    with open('links.txt', 'a') as file:
        file.write(link)
        file.write('\n')
        file.close()


In [13]:
df = date_expanded_dataset.copy(deep= True)
# df.iloc[:,3] = None


In [263]:
synopses_corrected_dataset

,اسم الفيلم,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,اليوم,الشهر,السنة,FirstName,ملخص,ملخص
0,يوم سعيد,???????,125.0,NaN,"محمد كريم,عبدالوارث عسر,عباس علام","محمد عبدالوهاب,علوية جميل,فاطمة محجوب,فاتن حما...",أفلام عبدالوهاب,"محمد عبدالعظيم,جورج بنوا,بريما فيرا,مصطفى حسن",محمد كريم,حسين رستم,NaN,"محمد عبدالوهاب,أحمد رامي,عزت الهجين,بشارة الخو...",محمد كريم,NaN,15,يناير,1940,NaN,محمد كمال شاب بسيط ويعطف عليه أصحاب البيت الذي...,محمد كمال شاب بسيط ويعطف عليه أصحاب البيت الذي...
1,الباشمقاول,??????,85.0,NaN,"بديع خيري,توجو مزراحي (أحمد المشرقي)","فوزي الجزايرلي,ميمي شكيب,إحسان الجزايرلي,زوزو ...","توجو مزراحي (أحمد المشرقي),عبدالحميد زكي","محمود نصر,عبدالحليم نصر",NaN,NaN,NaN,فريد غصن,"توجو مزراحي (أحمد المشرقي),إبراهيم حلمي",منتخبات بهنا فيلم,18,يناير,1940,NaN,NaN,NaN
2,تحت السلاح,nan,101.4,NaN,NaN,"أحمد علام,زوزو شكيب,عباس فارس,محمد الديب,زينات...",ألفيزى اورفانيللى,NaN,NaN,NaN,NaN,NaN,فؤاد الجزايرلي,NaN,8,فبراير,1940,NaN,NaN,NaN
3,قلب المرأة,???????,87.0,NaN,توجو مزراحي (أحمد المشرقي),"سليمان نجيب,أمينة رزق,دولت أبيض,عقيلة راتب,سلو...",NaN,عبدالحليم نصر,NaN,NaN,NaN,NaN,توجو مزراحي (أحمد المشرقي),منتخبات بهنا فيلم,6,مايو,1940,NaN,NaN,NaN
4,فتاة متمردة,nan,110.0,NaN,أحمد جلال,"ماري كويني,محسن سرحان,أنور وجدي,عباس فارس,بديع...",لوتس فيلم (آسيا وشركاها),"عبدالمنعم صبحى,توليو كيارينى",ماري كويني,NaN,NaN,"يوسف صالح,عبدالعزيز سلام,فريد غصن,ماري كويني,ش...","أحمد جلال,محمد زكى محمود",منتخبات بهنا فيلم,18,مايو,1940,NaN,NaN,NaN
5,رجل بين امراتين,nan,95.0,NaN,السيد زيادة,"بدر لاما,أمينة رزق,بدرية رأفت,فردوس محمد,عبدال...",كوندور فيلم,عبدالله الشيب,"ألبير نجيب,بدر لاما","ستوديو لاما,محروس زيادة",NaN,"يوسف صالح,سامي نعسان,أمينة رزق,بدر لاما,زوزو ش...",إبراهيم لاما,منتخبات بهنا فيلم,7,اكتوبر,1940,NaN,NaN,NaN
6,صرخة في الليل,nan,110.0,NaN,السيد زيادة,"بدر لاما,رجاء عبده,سميحة سميح,بشارة واكيم,منسى...",كوندور فيلم,إبراهيم شيبا,"بدر لاما,ألبير نجيب",محروس زيادة,NaN,"فريد غصن,يوسف صالح",إبراهيم لاما,منتخبات بهنا فيلم,31,اكتوبر,1940,NaN,NaN,NaN
7,الورشة,??ا??,105.0,NaN,"عزيزة أمير,محمود ذو الفقار","عزيزة أمير,محمود ذو الفقار,أنور وجدي,نجمة إبرا...",إيزيس فيلم,"أرام ماراليان,فيرى فاركاش (فرانسوا فاركاش)",NaN,محمد كامل,NaN,عبدالحميد عبدالرحمن,إستيفان روستي,منتخبات بهنا فيلم,28,نوفمبر,1940,NaN,NaN,NaN
8,أصحاب العقول,nan,101.4,NaN,NaN,"بهيجة المهدي,بشارة واكيم,فوزي منيب,سليمان نجيب",افلام وادى النيل (فهمي داود وشركاه),ألفيزى اورفانيللى,NaN,NaN,NaN,NaN,ألفيزى اورفانيللى,NaN,1,يناير,1940,NaN,NaN,NaN
9,حياة الظلام,??ا??,94.0,NaN,"محمود كامل حسن,أحمد بدرخان","ميمي شكيب,محسن سرحان,روحية خالد,فردوس محمد,علي...",محمد جمال الدين رفعت,محمد عبدالعظيم,جمال مدكور,"علي عابد,روبرت شارفنبرج",NaN,"بيرم التونسي,عبدالحميد عبدالرحمن,محمد الكحلاوي...","أحمد بدرخان,فريد الجندي",NaN,1,يناير,1940,NaN,NaN,NaN


In [452]:
with open('synopses.txt', 'r') as filehandle:
    
    for line in filehandle:
        
        current_line = line[:-1]
        line_content = current_line.split(':')
        movie_name_date = line_content[0].strip(')').split('(')
        name = movie_name_date[0]
        year = movie_name_date[1]
        plot = line_content[1]
        df[' ملخص'].loc[(df['اسم الفيلم'] == name) & (df['السنة'] == year)] = plot
        

In [464]:
movie_name ='اسير العيون'

df.loc[(df['اسم الفيلم'] == movie_name)]

,اسم الفيلم,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,اليوم,الشهر,السنة
323,اسير العيون,???????,87.0,شجاعة وحماسة وقوة جمعت بين قلوب الاحبة محمد و...,"أبو السعود الإبياري,بيرم التونسي,محمد الكحلاوي","محمد الكحلاوي,ليلى فوزي,علي الكسار,حورية محمد,...",NaN,NaN,NaN,NaN,NaN,NaN,إبراهيم حلمي,NaN,1,يناير,1949


In [15]:
with open('genres.txt', 'r') as filehandle:
    
    for line in filehandle:
        
        current_line = line[:-1]
        line_content = current_line.split(':')
        movie_name_date = line_content[0].strip(')').split('(')
        name = movie_name_date[0]
        year = movie_name_date[1]
        genre = line_content[1]
        
        df[' تصنيف الفيلم'].loc[(df['اسم الفيلم'] == name) & (df['السنة'] == year)]=genre

In [17]:
df.head(250)

,اسم الفيلم,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,اليوم,الشهر,السنة
0,يوم سعيد,ﺭﻭﻣﺎﻧﺴﻲ,125.0,(محمد كمال) شاب بسيط يهوى الموسيقى والغناء، وي...,"محمد كريم,عبدالوارث عسر,عباس علام","محمد عبدالوهاب,علوية جميل,فاطمة محجوب,فاتن حما...",أفلام عبدالوهاب,"محمد عبدالعظيم,جورج بنوا,بريما فيرا,مصطفى حسن",محمد كريم,حسين رستم,NaN,"محمد عبدالوهاب,أحمد رامي,عزت الهجين,بشارة الخو...",محمد كريم,NaN,15,يناير,1940
1,الباشمقاول,ﺩﺭاﻣﺎ,85.0,"تدور أحداث الفيلم حول المحامي ""ممتاز"" الذي يهو...","بديع خيري,توجو مزراحي (أحمد المشرقي)","فوزي الجزايرلي,ميمي شكيب,إحسان الجزايرلي,زوزو ...","توجو مزراحي (أحمد المشرقي),عبدالحميد زكي","محمود نصر,عبدالحليم نصر",NaN,NaN,NaN,فريد غصن,"توجو مزراحي (أحمد المشرقي),إبراهيم حلمي",منتخبات بهنا فيلم,18,يناير,1940
2,تحت السلاح,not available,101.4,أمينة فتاة من أسرة عريقة، تقرر العائلة زواجها ...,NaN,"أحمد علام,زوزو شكيب,عباس فارس,محمد الديب,زينات...",ألفيزى اورفانيللى,NaN,NaN,NaN,NaN,NaN,فؤاد الجزايرلي,NaN,8,فبراير,1940
3,قلب المرأة,ﺭﻭﻣﺎﻧﺴﻲ,87.0,تدور أحداث الفيلم حول (خيرية)، الفتاة الثرية ا...,توجو مزراحي (أحمد المشرقي),"سليمان نجيب,أمينة رزق,دولت أبيض,عقيلة راتب,سلو...",NaN,عبدالحليم نصر,NaN,NaN,NaN,NaN,توجو مزراحي (أحمد المشرقي),منتخبات بهنا فيلم,6,مايو,1940
4,فتاة متمردة,not available,110.0,درية طالبة فى مدرسة داخلية، تربطها علاقة حب بش...,أحمد جلال,"ماري كويني,محسن سرحان,أنور وجدي,عباس فارس,بديع...",لوتس فيلم (آسيا وشركاها),"عبدالمنعم صبحى,توليو كيارينى",ماري كويني,NaN,NaN,"يوسف صالح,عبدالعزيز سلام,فريد غصن,ماري كويني,ش...","أحمد جلال,محمد زكى محمود",منتخبات بهنا فيلم,18,مايو,1940
5,رجل بين امراتين,not available,95.0,يتعرف رافت الرسام على امال عبد الشافى وهو يصطا...,السيد زيادة,"بدر لاما,أمينة رزق,بدرية رأفت,فردوس محمد,عبدال...",كوندور فيلم,عبدالله الشيب,"ألبير نجيب,بدر لاما","ستوديو لاما,محروس زيادة",NaN,"يوسف صالح,سامي نعسان,أمينة رزق,بدر لاما,زوزو ش...",إبراهيم لاما,منتخبات بهنا فيلم,7,اكتوبر,1940
6,صرخة في الليل,not available,110.0,لا تحفظ الزوجة الساقطة شرف زوجها الثرى فترتبط ...,السيد زيادة,"بدر لاما,رجاء عبده,سميحة سميح,بشارة واكيم,منسى...",كوندور فيلم,إبراهيم شيبا,"بدر لاما,ألبير نجيب",محروس زيادة,NaN,"فريد غصن,يوسف صالح",إبراهيم لاما,منتخبات بهنا فيلم,31,اكتوبر,1940
7,الورشة,ﺩﺭاﻣﺎ,105.0,يسافر الأسطى (علي عبدالرحمن) صاحب ورشة الميكان...,"عزيزة أمير,محمود ذو الفقار","عزيزة أمير,محمود ذو الفقار,أنور وجدي,نجمة إبرا...",إيزيس فيلم,"أرام ماراليان,فيرى فاركاش (فرانسوا فاركاش)",NaN,محمد كامل,NaN,عبدالحميد عبدالرحمن,إستيفان روستي,منتخبات بهنا فيلم,28,نوفمبر,1940
8,أصحاب العقول,not available,101.4,كتب أحد المستكشفين الأوربيين إلى صديقه حمدي بك...,NaN,"بهيجة المهدي,بشارة واكيم,فوزي منيب,سليمان نجيب",افلام وادى النيل (فهمي داود وشركاه),ألفيزى اورفانيللى,NaN,NaN,NaN,NaN,ألفيزى اورفانيللى,NaN,1,يناير,1940
9,حياة الظلام,ﺩﺭاﻣﺎ,94.0,عائلة متوسطة الحال مكونة من زوج وزوجة وابنهما ...,"محمود كامل حسن,أحمد بدرخان","ميمي شكيب,محسن سرحان,روحية خالد,فردوس محمد,علي...",محمد جمال الدين رفعت,محمد عبدالعظيم,جمال مدكور,"علي عابد,روبرت شارفنبرج",NaN,"بيرم التونسي,عبدالحميد عبدالرحمن,محمد الكحلاوي...","أحمد بدرخان,فريد الجندي",NaN,1,يناير,1940


In [20]:
df.iloc[26, 3]

'يصل لعزيز تلغراف كاذب بوفاة والده عبد الغفار باشا، وفي القطار يتقابل مع فكرية ابنة العدو اللدود لوالده ، يعرف حقيقة استدعائه للبلدة حتى يناصر والده في حملته العدائية لأبي المجد والد فكرية إلا أنه يحاول الصلح بين الطرفين، وفي نفس الوقت يحاول التقرب من فكرية وتتوالى الأحداث.'

In [21]:
df.iloc[25,3]

'تدور قصة الفيلم حول سيدة متزوجة ولديها أبناء تكتشف خيانة زوجها لها، وكذلك يحدث الأمر مع زوجات أصدقاء زوجها، فتقررن جميعا هجر فراش الزوجية تأديبا لأزواجهن، ذلك الأمر الذي يجدي نفعا معهن حيث يعود كل زوج إلى زوجته مجددا بعد تركه للخيانة.'